# Week 2 Day 2
## YouTube Channel Manager Agent System
### System Overview
#### Component Description
1. Main Agent (channel_manager)	Triages requests like planning, scripting, emailing
2. Utility Agents topic_agent, planner_agent, script_agent, seo_agent — act as tools
3. Email Agent	Sends actual emails via Mailgun
4. Session	Remembers full conversation context
5. Tracing	Logs which agent/tool handled each request (Check Here: https://platform.openai.com/traces)

In [18]:
import os
import asyncio
import requests
from dotenv import load_dotenv
from agents import Agent, Runner, trace, SQLiteSession, function_tool
from openai.types.responses import ResponseTextDeltaEvent

In [19]:
topic_agent_instructions = "You are a YouTube Channel Topic Generator Agent. You will need to generate a topic that is relevant to the channel and that is not already in the channel's video titles."
topic_agent = Agent(name="topic_agent", instructions=topic_agent_instructions)

planner_agent_instructions = "You are a YouTube Channel Planner Agent. You plan YouTube videos for a channel. You will be given a topic and you will need to plan a video for that topic by breaking the topic into short actionable steps."
planner_agent = Agent(name="planner_agent", instructions=planner_agent_instructions)

script_agent_instructions = "You are a YouTube Channel Script Writer Agent. You will be given a topic. You will need to write an engaging script for a video for that topic and format."
script_agent = Agent(name="script_agent", instructions=script_agent_instructions)

seo_agent_instructions = "You are a YouTube Channel SEO Agent. You will be give a topic and you will need to write a SEO friendly title, description, tags, thumbnails, and a thumbnail description for that topic."
seo_agent = Agent(name="seo_agent", instructions=seo_agent_instructions)

email_agent_instructions = "You are a YouTube Channel Email Agent. You will be given a topic and you will need to send an email to the channel collaborators about the video plan, script, publishing events, and SEO."
email_agent = Agent(name="email_agent", instructions=email_agent_instructions)

In [20]:
channel = "Startup Building"

In [ ]:
result = await Runner.run(topic_agent, channel)
print(result.final_output)

In [ ]:
# Refer https://openai.github.io/openai-agents-python/streaming/

result = Runner.run_streamed(topic_agent, input=channel)
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
result = await Runner.run(planner_agent, channel)
print(result.final_output)

In [ ]:
result = await Runner.run(script_agent, channel)
print(result.final_output)

In [ ]:
result = await Runner.run(seo_agent, channel)
print(result.final_output)

In [ ]:


with trace("Putting together a YouTube Channel"):
    topic_for_video = await Runner.run(topic_agent, input=channel)
    topic_name = topic_for_video.final_output
    results = await asyncio.gather(
        Runner.run(planner_agent, input=topic_name),
        Runner.run(script_agent, input=topic_name),
        Runner.run(seo_agent, input=topic_name))

outputs = [result.final_output for result in results]

for output in outputs:
    print(f"Output: {output}")

## Create a function into a Tool

Simply wrap function with the decorator @function_tool

In [21]:
# MailGun API to send email notifications
from agents import function_tool
from typing import Annotated


@function_tool
def send_mailgun_email_notification(
    subject: Annotated[str, "The email subject line"],
    body: Annotated[str, "The email body content"]
) -> str:
    """
    Sends an email using Mailgun with the given subject and body.
    
    Args:
        subject: The email subject line
        body: The email body content
        
    Returns:
        A string indicating success or failure of the email send operation
    """
    response = requests.post(
        f"https://api.mailgun.net/v3/{os.getenv('MAIL_GUN_DOMAIN')}/messages",
        auth=("api", os.getenv('MAIL_GUN_API_KEY')),
        data={
            "from": os.getenv('MAIL_GUN_YOUTUBE_CHANNEL_FROM_EMAIL'),
            "to": os.getenv('MAIL_GUN_YOUTUBE_CHANNEL_TO_EMAIL'),
            "subject": subject,
            "text": body
        }
    )

    if response.status_code == 200:
        return "Email sent successfully."
    else:
        return f"Email failed: {response.text}"

In [ ]:


print(send_mailgun_email_notification.params_json_schema)

# Convert an Agent into a tool

Simply use `.as_tool()`

In [ ]:
topic_agent_as_tool = topic_agent.as_tool(tool_name="topic_agent_tool", tool_description="A tool that generates a topic for a YouTube video")

topic_agent_as_tool

In [ ]:
planner_agent_as_tool = planner_agent.as_tool(tool_name="planner_agent_tool", tool_description="A tool that plans a YouTube video")
script_agent_as_tool = script_agent.as_tool(tool_name="script_agent_tool", tool_description="A tool that writes a script for a YouTube video")
seo_agent_as_tool = seo_agent.as_tool(tool_name="seo_agent_tool", tool_description="A tool that writes a SEO friendly title, description, tags, thumbnails, and a thumbnail description for a YouTube video")

utility_tools = [topic_agent_as_tool, planner_agent_as_tool, script_agent_as_tool, seo_agent_as_tool]
utility_tools

In [ ]:
email_notification_agent_instructions = """
You are a YouTube Channel Email Agent. 
You are responsible for sending email notifications. 
Use the send_mailgun_email_notification tool."""

# email_tools = [send_mailgun_email_notification]
email_notification_agent = Agent(name="email_notification_agent", instructions=email_notification_agent_instructions, tools=[send_mailgun_email_notification])

handoff_tools = [email_notification_agent]
handoff_tools

## Main Agent that will handle all the management for your YouTube Channel's topic

In [ ]:
channel_manager_agent_instructions = """
You are a YouTube Channel Manager Agent.

You will be given a video topic for a YouTube channel. Your responsibilities:

1. Plan the video by breaking the topic into clear, actionable steps.
2. Write an engaging, high-retention script tailored to the format.
3. Generate SEO assets: a title, description, tags, and thumbnail with description.

Once you've created the video plan and content:

4. You must send an email to the channel collaborators summarizing the video plan, script, publishing timeline, and SEO assets.

To do this: 
1. You must call the `email` agent via handoff, passing exactly two arguments: the `subject` and `body` of the email.  
2. Do not print or include the email in your final response.  
3. Do not ask for permission to send it.  
4. Just trigger the handoff with the final subject and body.

Use each tool only once. Use all tools effectively. Do not repeat yourself. Be specific and concise.
"""


channel_manager_agent = Agent(name="channel_manager_agent", instructions=channel_manager_agent_instructions, tools=utility_tools, handoffs={"email": email_notification_agent})

channel_manager_agent


In [28]:
channel_session = SQLiteSession("channel_session")

In [ ]:
with trace("Agentic AI System for YouTube Channel Management"):
    result = await Runner.run(channel_manager_agent, channel, session=channel_session)

print(f"Channel Manager Agent Output: {result.final_output}")